找到具有最小间隔的数据点，再对该间隔最大化：
$$
\mathop{\arg\max}_{(w,b)}\{\min_n \ label·\frac{(w^Tx+b)}{||w||}\}
$$
转化后的优目标函数：
$$
\begin{align}
\max_\alpha[\sum_{i=1}^m\alpha-\frac{1}{2}\sum_{i,j=1}^{m}&label^{(i)}·label^{(j)}·\alpha_i·\alpha_j·<x^{(i)},x^{(j)}>] \\\\
st. \ &\alpha\geq0\\
&\sum_{i-1}^m\alpha_i·label^{(i)}=0
\end{align}
$$
引入松弛变量后约束条件变为：
$$
\begin{align}
st. \ &C\geq\alpha\geq0\\&\sum_{i-1}^m\alpha_i·label^{(i)}=0
\end{align}
$$


**SMO：序列最小优化（Sequential Minimal Optimization）**
将大优化问题分解为多个小优化问题  
SMO目标：求出一系列$\alpha$和$b$，然后可以容易计算出$w$  
工作原理：  
1.每次循环选择两个$\alpha$进行优化处理  
2.一旦找到一对“合适”的$\alpha$，就一个增大一个减小  

In [1]:
import numpy as np

In [2]:
# 读取数据，获得特征与标签
def loadDataSet(fileName):
    dataMat = []
    labelMat = []
    fr = open(fileName)
    for line in fr.readlines():
        lineArr = line.strip().split('\t')
        dataMat.append([float(lineArr[0]), float(lineArr[1])])
        labelMat.append(float(lineArr[2]))
    return dataMat, labelMat

# i是alpha下标，m是alpha数目
def selectJrand(i, m):
    j = i;
    while j == i:
        j = int(np.random.uniform(0, m))
    return j

# 调整大于H或小于L的alpha值
def clipAlpha(aj, H, L):
    if aj > H:
        aj = H
    if L > aj:
        aj = L
    return aj

## 简化版SMO算法
**伪代码**:  
创建一个alpha向量并将其初始化为0向量  
当迭代次数小于最大迭代次数时（外循环）  
&ensp;&ensp;&ensp;&ensp;对数据集中的每个数据向量（内循环）：  
&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;如果该数据向量可以被优化：  
&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;随机选择另外一个数据向量  
&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;同时优化这两个向量  
&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;如果两个向量都不能被优化，退出内循环  
&ensp;&ensp;&ensp;&ensp;如果所有向量都没被优化，增加迭代数目，继续下一次循环

In [3]:
# 参数：数据集、类别标签、常数C、容错率、循环最大次数
def smoSimple(dataMatIn, classLabels, C, toler, maxIter):
    dataMatrix = np.mat(dataMatIn)
    labelMat = np.mat(classLabels).transpose()
    b = 0
    m,n = np.shape(dataMatrix)
    alphas = np.mat(np.zeros((m, 1)))    # 初始化为0向量
    iter = 0
    # 当迭代次数小于最大迭代次数
    while iter < maxIter:
        alphaPairsChanged = 0    # 记录alpha是否已经优化
        # 对数据集中的每个数据向量
        for i in range(m):
            fXi = float(np.multiply(alphas, labelMat).T * (dataMatrix * dataMatrix[i, :].T)) + b    # 预测类别
            Ei = fXi - float(labelMat[i])    # 误差
            # 如果alpha可以更改，进入优化过程
            if((labelMat[i] * Ei < -toler) and (alphas[i] < C)) or ((labelMat[i] * Ei > toler) and (alphas[i] > 0)):
                # 随机选择另外一个数据向量
                j = selectJrand(i, m)
                fXj = float(np.multiply(alphas, labelMat).T * (dataMatrix * dataMatrix[j, :].T)) + b
                Ej = fXj - float(labelMat[j])    # 计算第二个alpha的误差
                alphaIold = alphas[i].copy()
                alphaJold = alphas[j].copy()
                # 保证alpha在0与C之间
                if labelMat[i] != labelMat[j]:
                    L = max(0, alphas[j] - alphas[i])
                    H = min(C, C + alphas[j] - alphas[i])
                else:
                    L = max(0, alphas[j] - alphas[i] - C)
                    H = min(C, alphas[j] - alphas[i])
                if L == H:
                    print('L == H')
                    continue
                # alpha[j]的最优修改量
                eta = 2.0 * dataMatrix[i, :] * dataMatrix[j, :].T - dataMatrix[i, :] * dataMatrix[i, :].T - dataMatrix[j, :] * dataMatrix[j, :].T
                if eta >= 0:
                    print('eta >= 0')
                    continue
                # 计算新的alpha[j]
                alphas[j] -= labelMat[j] * (Ei - Ej) / eta
                alphas[j] = clipAlpha(alphas[j], H, L)
                # 检查alpha[j]是否有轻微改变
                if abs(alphas[j] - alphaJold) < 0.00001:
                    print('j not moving enough')
                    continue
                # 对i进行修改，修改量与j相同，但方向相反
                alphas[i] += labelMat[j] * labelMat[i] * (alphaJold - alphas[j])
                # 设置常数项b
                b1 = b - Ei - labelMat[i] * (alphas[i] - alphaIold) * dataMatrix[i, :] * dataMatrix[i].T - labelMat[j] * (alphas[j] - alphaJold) * dataMatrix[i, :] * dataMatrix[j, :].T
                b2 = b - Ej - labelMat[i] * (alphas[i] - alphaIold) * dataMatrix[i, :] * dataMatrix[j].T - labelMat[j] * (alphas[j] - alphaJold) * dataMatrix[j, :] * dataMatrix[j, :].T
                if 0 < alphas[i] and C > alphas[i]:
                    b = b1
                elif 0 < alphas[j] and C > alphas[j]:
                    b = b2
                else:
                    b = (b1 + b2)/2.0
                alphaPairsChanged += 1    # 成功改变
                print('iter: %d  i: %d, pairs changed %d' % (iter, i, alphaPairsChanged))
        # 检查是否有更新alpha，若有，则将iter设为0继续
        if alphaPairsChanged ==0:
            iter += 1
        else:
            iter = 0
        print('iteration number: %d' % iter)
    return b, alphas

In [4]:
dataArr, labelArr = loadDataSet('testSet.txt')
b, alphas = smoSimple(dataArr, labelArr, 0.6, 0.001, 40)

iter: 0  i: 0, pairs changed 1
j not moving enough
j not moving enough
iter: 0  i: 5, pairs changed 2
L == H
L == H
iter: 0  i: 10, pairs changed 3
L == H
j not moving enough
j not moving enough
iter: 0  i: 17, pairs changed 4
j not moving enough
L == H
iter: 0  i: 23, pairs changed 5
j not moving enough
L == H
iter: 0  i: 31, pairs changed 6
j not moving enough
j not moving enough
L == H
L == H
L == H
iter: 0  i: 55, pairs changed 7
j not moving enough
j not moving enough
j not moving enough
L == H
j not moving enough
L == H
L == H
j not moving enough
j not moving enough
iteration number: 0
j not moving enough
j not moving enough
j not moving enough
L == H
j not moving enough
L == H
j not moving enough
j not moving enough
j not moving enough
L == H
j not moving enough
L == H
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
L == H
L == H
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough

iteration number: 0
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iter: 0  i: 23, pairs changed 1
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number: 0
j not moving enough
j not moving enough
j not moving enough
iter: 0  i: 17, pairs changed 1
j not moving enough
j not moving enough
j not moving enough
iter: 0  i: 54, pairs changed 2
j not moving enough
j not moving enough
j not moving enough
L == H
L == H
iter: 0  i: 92, pairs changed 3
iteration number: 0
L == H
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iter: 0  i: 17, pairs changed 1
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iter: 0  i: 54, pairs changed 2
j not moving enough
L == H
j not moving enough
j not moving enough
iteration number: 0
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iter: 0  i: 29, pairs changed 1
j not mo

j not moving enough
iteration number: 0
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iter: 0  i: 69, pairs changed 1
j not moving enough
iteration number: 0
j not moving enough
j not moving enough
j not moving enough
iter: 0  i: 54, pairs changed 1
j not moving enough
iter: 0  i: 69, pairs changed 2
j not moving enough
iteration number: 0
j not moving enough
iter: 0  i: 23, pairs changed 1
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number: 0
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number: 1
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number: 2
j not moving enough
j not moving enough
j not moving enough
iter: 2  i: 54, pairs changed 1
L == H
j not moving enough
j not moving enough
iteration num

j not moving enough
j not moving enough
j not moving enough
iteration number: 10
j not moving enough
j not moving enough
j not moving enough
iteration number: 11
j not moving enough
j not moving enough
iter: 11  i: 54, pairs changed 1
j not moving enough
iteration number: 0
j not moving enough
j not moving enough
j not moving enough
iteration number: 1
j not moving enough
j not moving enough
j not moving enough
iteration number: 2
j not moving enough
j not moving enough
j not moving enough
iteration number: 3
j not moving enough
j not moving enough
j not moving enough
iteration number: 4
j not moving enough
j not moving enough
iter: 4  i: 55, pairs changed 1
iteration number: 0
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number: 1
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number: 2
j not moving enough
j not moving enough
iter: 2  i: 52, pairs changed 1
j not moving enough
j not moving enough
L

iteration number: 9
j not moving enough
j not moving enough
L == H
iteration number: 10
j not moving enough
j not moving enough
L == H
iteration number: 11
iter: 11  i: 29, pairs changed 1
j not moving enough
j not moving enough
j not moving enough
j not moving enough
L == H
iteration number: 0
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
L == H
iteration number: 1
j not moving enough
iter: 1  i: 52, pairs changed 1
j not moving enough
j not moving enough
iteration number: 0
j not moving enough
j not moving enough
j not moving enough
iteration number: 1
j not moving enough
j not moving enough
j not moving enough
iteration number: 2
j not moving enough
j not moving enough
j not moving enough
iteration number: 3
j not moving enough
j not moving enough
j not moving enough
iteration number: 4
j not moving enough
j not moving enough
j not moving enough
iteration number: 5
j not moving enough
j not moving enough
j not moving enough
itera

j not moving enough
j not moving enough
L == H
j not moving enough
j not moving enough
iteration number: 1
j not moving enough
j not moving enough
L == H
j not moving enough
L == H
iteration number: 2
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iter: 2  i: 70, pairs changed 1
j not moving enough
j not moving enough
iteration number: 0
j not moving enough
j not moving enough
L == H
j not moving enough
j not moving enough
j not moving enough
iteration number: 1
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number: 2
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
L == H
iteration number: 3
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iter: 3  i: 69, pairs changed 1
j not moving enough
j

j not moving enough
j not moving enough
iteration number: 27
j not moving enough
j not moving enough
L == H
iteration number: 28
j not moving enough
j not moving enough
L == H
iteration number: 29
j not moving enough
j not moving enough
L == H
iteration number: 30
j not moving enough
j not moving enough
j not moving enough
iteration number: 31
j not moving enough
j not moving enough
L == H
iteration number: 32
j not moving enough
j not moving enough
L == H
iteration number: 33
j not moving enough
j not moving enough
j not moving enough
iteration number: 34
j not moving enough
j not moving enough
L == H
iteration number: 35
j not moving enough
j not moving enough
j not moving enough
iteration number: 36
j not moving enough
j not moving enough
L == H
iteration number: 37
j not moving enough
j not moving enough
j not moving enough
iteration number: 38
j not moving enough
j not moving enough
L == H
iteration number: 39
j not moving enough
j not moving enough
j not moving enough
iteration n

In [5]:
b

matrix([[-4.97599549]])

In [6]:
alphas[alphas>0]

matrix([[1.21430643e-17, 4.16799009e-01, 4.16799009e-01]])

In [7]:
# 支持向量的个数
np.shape(alphas[alphas>0])

(1, 3)

In [8]:
# 查看支持向量
for i in range(100):
    if alphas[i] > 0.0:
        print(dataArr[i], labelArr[i])

[2.074915, 1.41055] -1.0
[2.893743, -1.643468] -1.0
[5.286862, -2.358286] 1.0
